# Sending and Receiving Cookies

In [10]:
import requests

custom_cookie = {"user_id": "2"}

response = requests.get(
    "http://127.0.0.1:8000/api/cookies", 
    cookies=custom_cookie
)

print(response.cookies.get_dict())
print(response.cookies["user_id"])


DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:8000
DEBUG:urllib3.connectionpool:http://127.0.0.1:8000 "GET /api/cookies HTTP/1.1" 200 16


{'user_id': '2'}
2


# Persisting Connections with Sessions

In [13]:
import requests

with requests.Session() as session:
    credentials = {"username": "some_name", "password": "pass"}
    
    session.post(
        "http://127.0.0.1:8000/api/login",
        data=credentials
    )

    response = session.get(
        "http://127.0.0.1:8000/protected"
    )

    print("Protected route:")
    print(response.status_code)
    print(response.text)
                                  

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:8000
DEBUG:urllib3.connectionpool:http://127.0.0.1:8000 "POST /api/login HTTP/1.1" 200 30
DEBUG:urllib3.connectionpool:http://127.0.0.1:8000 "GET /protected HTTP/1.1" 200 53


Protected route:
200
{"message":"You have access to this protected route"}


# Retrying Failed Connections with Transport Adapters

In [14]:
import logging
import requests
from requests.adapters import HTTPAdapter
from requests.exceptions import RetryError
from urllib3.util.retry import Retry    

logging.basicConfig(level=logging.DEBUG)
request_log = logging.getLogger("urllib3")
request_log.setLevel(logging.DEBUG)
request_log.propagate = True

session = requests.Session()
retries = Retry(total=3, backoff_factor=0.1, status_forcelist=[500], allowed_methods={"GET"})
session.mount("http://127.0.0.1", HTTPAdapter(max_retries=retries))

try:
    response = session.get("http://127.0.0.1:8000/flaky")
    print("Final response:", response.status_code)

except RetryError as e:
    print("Maximal number of retries exceeded. Server seems to be down.")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:8000
DEBUG:urllib3.connectionpool:http://127.0.0.1:8000 "GET /flaky HTTP/1.1" 200 21


Final response: 200
